<a href="https://colab.research.google.com/github/MackPreston/iot-traffic-analysis/blob/main/IoT_Traffic_Analysis_Tranalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IoT Traffic Analysis with Tranalyzer Flows

In [1]:
import sys
import os
import glob
print(sys.version_info)
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import collections

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,classification_report

sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)


## Loading benign dataset

The dataset is then loadeded from tranalyzers output into a dataframe

In [36]:
print("Current Directory:" + os.getcwd())
# file_pattern = '*_flows.txt' # Use all pcap files
file_pattern = '18-05-29_flows.txt'
glob_path = os.path.join(os.getcwd(), 'benign/tran', file_pattern)
file_list = glob.glob(glob_path)

li = []
for filepath in file_list:
    df = pd.read_csv(filepath, index_col=None, header=0, delimiter='\t')
    li.append(df)

benign_flows = pd.concat(li, axis=0, ignore_index=True)
benign_flows

Current Directory:/mnt/e/iot-traffic-analysis


/home/mack/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,%dir,flowInd,flowStat,timeFirst,timeLast,duration,numHdrDesc,numHdrs,hdrDesc,srcMac,...,icmpTCcnt,icmpBFTypH_TypL_Code,icmpTmGtw,icmpEchoSuccRatio,icmpPFindex,connSip,connDip,connSipDip,connSipDprt,connF
0,A,2,0x0400000000004000,1.527552e+09,1.527552e+09,0.012182,1,3,eth:ipv4:tcp,14:cc:20:51:33:ea,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,1,1,2,2,2.0
1,B,2,0x0400000000004001,1.527552e+09,1.527552e+09,0.001862,1,3,eth:ipv4:tcp,ec:1a:59:83:28:11,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,1,1,1,1,1.0
2,A,3,0x0400000000004000,1.527552e+09,1.527552e+09,2.146254,1,3,eth:ipv4:tcp,ec:1a:59:83:28:11,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,1,1,2,2,2.0
3,B,3,0x0400000000004001,1.527552e+09,1.527552e+09,1.922308,1,3,eth:ipv4:tcp,14:cc:20:51:33:ea,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,1,1,1,1,1.0
4,A,24,0x0400000000004000,1.527552e+09,1.527552e+09,0.012025,1,3,eth:ipv4:tcp,ec:1a:59:83:28:11,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,5,1,4,4,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90926,B,47443,0x0400000000004001,1.527635e+09,1.527638e+09,3514.705441,1,3,eth:ipv4:tcp,14:cc:20:51:33:ea,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,1,1,1,1,1.0
90927,A,49217,0x0400000000004000,1.527638e+09,1.527638e+09,45.225318,1,3,eth:ipv4:tcp,f4:f5:d8:d4:eb:12,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,1,1,2,2,2.0
90928,B,49217,0x0400000000004001,1.527638e+09,1.527638e+09,45.224967,1,3,eth:ipv4:tcp,14:cc:20:51:33:ea,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,1,1,1,1,1.0
90929,A,1,0x0400000000004000,1.527552e+09,1.527638e+09,86397.810988,1,3,eth:ipv4:tcp,30:8c:fb:2f:e4:b2,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,1,1,2,2,2.0


# Loading Mixed Dataset

In [3]:
import os
import glob
print("Current Directory:" + os.getcwd())
# file_pattern = '*_flows.txt' # Use all pcap files
file_pattern = '18-06-01_flows.txt'
glob_path = os.path.join(os.getcwd(), 'attack/tran', file_pattern)
file_list = glob.glob(glob_path)

li = []
for filepath in file_list:
    df = pd.read_csv(filepath, index_col=None, header=0, delimiter='\t')
    li.append(df)

attack_flows = pd.concat(li, axis=0, ignore_index=True)
attack_flows

Current Directory:/mnt/e/iot-traffic-analysis


/home/mack/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,%dir,flowInd,flowStat,timeFirst,timeLast,duration,numHdrDesc,numHdrs,hdrDesc,srcMac,...,icmpTCcnt,icmpBFTypH_TypL_Code,icmpTmGtw,icmpEchoSuccRatio,icmpPFindex,connSip,connDip,connSipDip,connSipDprt,connF
0,A,1,0x0400000000004000,1.527823e+09,1.527823e+09,0.190275,1,3,eth:ipv4:tcp,e0:76:d0:3f:00:ae,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,1,1,4,4,4.0
1,B,1,0x0400000000004001,1.527823e+09,1.527823e+09,0.000187,1,3,eth:ipv4:tcp,14:cc:20:51:33:ea,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,1,1,3,3,3.0
2,A,2,0x0400000000004000,1.527823e+09,1.527823e+09,10.196861,1,3,eth:ipv4:tcp,e0:76:d0:3f:00:ae,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,3,1,4,6,2.0
3,B,2,0x0400000000004001,1.527823e+09,1.527823e+09,10.007584,1,3,eth:ipv4:tcp,14:cc:20:51:33:ea,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,1,3,3,5,5.0
4,A,7,0x0400000000004000,1.527823e+09,1.527823e+09,10.196851,1,3,eth:ipv4:tcp,e0:76:d0:3f:00:ae,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,5,1,2,8,1.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226479,B,117860,0x0400000000004001,1.527897e+09,1.527897e+09,0.219089,1,3,eth:ipv4:tcp,14:cc:20:51:33:ea,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,1,1,1,1,1.0
226480,A,120,0x0400000000004000,1.527823e+09,1.527897e+09,74067.670764,1,3,eth:ipv4:tcp,30:8c:fb:2f:e4:b2,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,1,1,2,2,2.0
226481,B,120,0x0400000000004001,1.527823e+09,1.527897e+09,74066.088314,1,3,eth:ipv4:tcp,14:cc:20:51:33:ea,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,1,1,1,1,1.0
226482,A,117861,0x0800000000008000,1.527897e+09,1.527897e+09,0.000000,1,3,eth:ipv6:udp,e0:76:d0:3f:00:ae,...,0,0x00000000_0x00000000_0x0000,0x00000000,0.0,0,1,1,1,1,1.0


# Feature Selection

In [87]:
# Features which will not be used
pointless_features = ['timeFirst', 'timeLast', 'dstPort', 'srcPort', 'dstIP', 
                      'dstMac', 'srcIP', 'srcMac', 'srcMac_dstMac_numP']

# Non-decimal features categorized by data-type
hex_features = ['flowStat', 'tcpFStat', 'ipTOS', 'ipFlags', 'ethType',
                 'tcpStates', 'icmpStat', 'icmpTmGtw', 'macStat','tcpAnomaly', 
                'tcpFlags',  'tcpMPF', 'tcpMPTBF', 'tcpMPdssF', 'tcpOptions']


string_features = ['%dir', 'hdrDesc', 'srcManuf_dstManuf', 'dstPortClass', 
                   'srcIPCC', 'dstIPCC', 'dstIPOrg', 'srcIPOrg']

# Not sure what to make of these - more research required
underscore_features = ['ipOptCpCl_Num', 'icmpBFTypH_TypL_Code', 'ip6OptHH_D', 'ip6OptCntHH_D']

# Features we will use
features = ['flowInd', 'duration', 'numHdrDesc', 'numHdrs',
            'ethVlanID', 'l4Proto', 'macPairs', 'dstPortClassN', 'numPktsSnt', 'numPktsRcvd',
            'numBytesSnt', 'numBytesRcvd', 'minPktSz', 'maxPktSz', 'avePktSize', 'stdPktSize', 'pktps', 'bytps',
            'pktAsm', 'bytAsm', 'ipMindIPID', 'ipMaxdIPID', 'ipMinTTL', 'ipMaxTTL', 'ipTTLChg', 'ipOptCnt',
            'tcpPSeqCnt', 'tcpSeqSntBytes', 'tcpSeqFaultCnt', 'tcpPAckCnt', 'tcpFlwLssAckRcvdBytes', 'tcpAckFaultCnt',
            'tcpInitWinSz', 'tcpAveWinSz', 'tcpMinWinSz', 'tcpMaxWinSz', 'tcpWinSzDwnCnt', 'tcpWinSzUpCnt',
            'tcpWinSzChgDirCnt', 'tcpOptPktCnt', 'tcpOptCnt', 'tcpMSS', 'tcpWS', 'tcpTmS', 'tcpTmER', 'tcpEcI',
            'tcpBtm', 'tcpSSASAATrip', 'tcpRTTAckTripMin', 'tcpRTTAckTripMax', 'tcpRTTAckTripAve',
            'tcpRTTAckTripJitAve', 'tcpRTTSseqAA', 'tcpRTTAckJitAve', 'icmpTCcnt', 'icmpEchoSuccRatio', 'icmpPFindex',
            'connSip', 'connDip', 'connSipDip', 'connSipDprt', 'connF',  'aveIAT', 'maxIAT', 
                'minIAT', 'stdIAT', 'tcpISeqN', 'tcpMPAID', 'tcpUtm', 'tcpWinSzThRt']

unused_features = pointless_features + hex_features + string_features + underscore_features
all_features = features + unused_features

benign_flows[features]


,flowInd,duration,numHdrDesc,numHdrs,ethVlanID,l4Proto,macPairs,dstPortClassN,numPktsSnt,numPktsRcvd,...,connSipDprt,connF,aveIAT,maxIAT,minIAT,stdIAT,tcpISeqN,tcpMPAID,tcpUtm,tcpWinSzThRt
0,2,0.012182,1,3,NaN,6,1,3483,4,2,...,2,2.0,0.003046,0.011954,0,0.003860,2812776272,0,1.184785e+07,0.0
1,2,0.001862,1,3,NaN,6,1,3483,2,4,...,1,1.0,0.000931,0.001862,0,0.000658,3531972144,0,8.661374e+03,0.0
2,3,2.146254,1,3,NaN,6,1,8443,18,16,...,2,2.0,0.119236,0.513567,0,0.128981,3527834515,0,3.464771e+04,0.0
3,3,1.922308,1,3,NaN,6,1,8443,16,18,...,1,1.0,0.120144,0.738684,0,0.171620,1185320615,0,1.184786e+07,0.0
4,24,0.012025,1,3,NaN,6,1,49152,5,5,...,4,0.8,0.002405,0.005367,0,0.001676,3545832534,0,8.663462e+03,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90926,47443,3514.705441,1,3,NaN,6,1,8883,506,496,...,1,1.0,6.946059,10.847980,0,4.088682,3710804030,0,0.000000e+00,0.0
90927,49217,45.225318,1,3,NaN,6,1,443,15,11,...,2,2.0,3.015021,45.035880,0,10.849790,1730083815,0,2.316842e+04,0.0
90928,49217,45.224967,1,3,NaN,6,1,443,11,15,...,1,1.0,4.111361,44.996220,0,12.327320,374328926,0,3.724168e+06,0.0
90929,1,86397.810988,1,3,NaN,6,1,443,2008267,1069619,...,2,2.0,0.043016,1.399033,0,0.056104,132537641,0,1.199746e+05,0.0
